In [2]:
import pandas as pd

## Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [3]:
attendance = pd.read_csv('attendance.csv')
attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [15]:
attendance = attendance.rename(columns={'Student': 'student'})
attendance.head()

,student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [35]:
attendance_melt = attendance.melt(id_vars=['student'],
               var_name= 'date',
               value_name= 'grade')

attendance_melt.head()

,student,date,grade
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [36]:
attendance_melt.grade.unique()

array(['P', 'A', 'T', 'H'], dtype=object)

In [37]:
gby = attendance_melt.groupby(attendance_melt.student)
gby

In [53]:
conv_grade = attendance_melt.grade.replace({'P': 1, 'A': 0, 'T':.9, 'H': .5})
conv_grade.head()

0    1.0
1    0.0
2    0.0
3    1.0
4    0.9
Name: grade, dtype: float64

In [40]:
attendance_melt.grade = conv_grade
attendance_melt.head()

,student,date,grade
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [41]:
attendance_melt.groupby('student').grade.mean()


student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

Coffee Levels
Read the coffee_levels table.

In [47]:
coffee_levels = pd.read_csv('coffee_levels.csv')
coffee_levels.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


Transform the data so that each carafe is in it's own column.

In [48]:
coffee_levels.coffee_carafe.value_counts()

x    10
y    10
z    10
Name: coffee_carafe, dtype: int64

In [50]:
transformed_df = coffee_levels.pivot(index='hour', columns='coffee_carafe', values='coffee_amount')
transformed_df

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


Is this the best shape for the data?

Cake Recipes

Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [54]:
cake = pd.read_csv('cake_recipes.csv')
cake.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


Tidy the data as necessary.

In [57]:
expanded = cake['recipe:position'].str.split(':', expand=True)
expanded.head()

,0,1
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom


In [62]:
cake = pd.concat([expanded, cake], axis=1)
cake = cake.drop(columns='recipe:position')
cake

,0,1,225,250,275,300
0,a,bottom,61.738655,53.912627,74.414730,98.786784
1,a,top,51.709751,52.009735,68.576858,50.228470
2,b,bottom,57.095320,61.904369,61.196980,99.248541
3,b,top,82.455004,95.224151,98.594881,58.169349
4,c,bottom,96.470207,52.001358,92.893227,65.473084
5,c,top,71.306308,82.795477,92.098049,53.960273
6,d,bottom,52.799753,58.670419,51.747686,56.183110
7,d,top,96.873178,76.101363,59.571620,50.971626


In [65]:
cake = cake.rename(columns={0:'recipe', 1:'position'})
cake.head()

,recipe,position,225,250,275,300
0,a,bottom,61.738655,53.912627,74.414730,98.786784
1,a,top,51.709751,52.009735,68.576858,50.228470
2,b,bottom,57.095320,61.904369,61.196980,99.248541
3,b,top,82.455004,95.224151,98.594881,58.169349
4,c,bottom,96.470207,52.001358,92.893227,65.473084


Which recipe, on average, is the best?

In [72]:
cake.columns

Index(['recipe', 'position', '225', '250', '275', '300'], dtype='object')

In [81]:
cake = cake.assign(avg_taste_rating= (cake['225'] + cake['250'] + cake['275'] + cake['300'])/ 4)

In [88]:
new_cake = cake.groupby('recipe').avg_taste_rating.mean()
new_cake.sort_values(ascending=False)

recipe
b    76.736074
c    75.874748
a    63.922201
d    62.864844
Name: avg_taste_rating, dtype: float64

Which oven temperature, on average, produces the best results?

In [94]:
print(f"The average taste score of 225 is {cake['225'].mean()}.")
print(f"The average taste score of 250 is {cake['250'].mean()}.")
print(f"The average taste score of 275 is {cake['275'].mean()}.")
print(f"The average taste score of 300 is {cake['300'].mean()}.")

The average taste score of 225 is 71.3060219226864.
The average taste score of 250 is 66.57743735826458.
The average taste score of 275 is 74.88675386649831.
The average taste score of 300 is 66.62765455006792.


Which combination of recipe, rack position, and temperature gives the best result?

In [100]:
melted_cake = cake.melt(id_vars=['recipe', 'position'],
             var_name= 'temperature',
             value_name= 'result')

sorted_cake = melted_cake.sort_values('result', ascending=False)

best_combination = sorted_cake.iloc[0]

best_combination

recipe                 b
position          bottom
temperature          300
result         99.248541
Name: 26, dtype: object